In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from utils import *

In [109]:
class TrafficGCNN(nn.Module):
    def __init__(self, num_nodes, num_edges, device='cpu'):
        super(TrafficGCNN, self).__init__()
        self.num_nodes = num_nodes
        self.num_edges = num_edges
        self.device = device
        
        # Graph convolution parameters
        self.theta = nn.Parameter(torch.abs(torch.randn(num_nodes, num_nodes, device=device)))
        
        # Flow transformation parameters
        self.W_q = nn.Parameter(torch.abs(torch.randn(num_edges, num_nodes, device=device)))
        self.W_F = nn.Parameter(torch.abs(torch.randn(num_edges, device=device)))

    def _get_link_capacities(self, C):
        """Convert node-to-node capacity matrix to link capacities"""
        batch_size = C.size(0)
        
        # 1. Create binary adjacency mask from capacity matrix
        adj_mask = (C > 0).float()  # [batch, N, N]
        
        # 2. Reshape to gather link capacities
        C_links = C.view(batch_size, -1)  # [batch, N*N]
        adj_mask = adj_mask.view(batch_size, -1)  # [batch, N*N]
        
        # 3. Filter only existing links (non-zero capacities)
        link_capacities = C_links[adj_mask.bool()].view(batch_size, self.num_edges)
        
        return link_capacities  # [batch, E]

    def forward(self, X, A_w, C):
        """
        Args:
            X: [batch_size, num_nodes, num_nodes] - OD matrix
            A_w: [batch_size, num_nodes, num_nodes] - Weighted adjacency
            C: [batch_size, num_nodes, num_nodes] - Capacity matrix (0 where no links)
        Returns:
            F: [batch_size, num_edges] - Link flows
        """
        batch_size = X.size(0)
        
        # 1. Convert capacity matrix to link capacities
        C_links = self._get_link_capacities(C)  # [batch, E]
        
        # 2. Graph convolution
        I = torch.eye(self.num_nodes, device=self.device).expand(batch_size, -1, -1)
        A_w_bar = A_w + I
        D_w_bar = torch.diag_embed(A_w_bar.sum(dim=-1))
        D_w_inv = torch.linalg.pinv(D_w_bar)
        transition_matrix = torch.bmm(D_w_inv, A_w_bar)
        
        theta_batch = torch.abs(self.theta).unsqueeze(0).expand(batch_size, -1, -1)
        H1 = torch.tanh(torch.bmm(theta_batch, torch.bmm(transition_matrix, X)))
        
        # 3. Flow distribution
        W_q_batch = torch.abs(self.W_q).unsqueeze(0).expand(batch_size, -1, -1)
        H2 = torch.tanh(torch.bmm(W_q_batch, H1))  # [batch, E, N]
        
        # 4. Output aggregation
        W_F_batch = torch.abs(self.W_F).unsqueeze(0).expand(batch_size, -1)  # [batch, E]
        F = torch.sum(H2 * W_F_batch.unsqueeze(-1), dim=2)  # [batch, E]
        F = torch.nn.functional.softplus(F)  # [batch, E]
        
        # 5. Apply capacity constraints
        F = torch.min(F, C_links)  # Both [batch, E]
        return F

class TrafficAssignmentModel:
    def __init__(self, num_nodes, num_edges, device='cpu'):
        self.gcnn = TrafficGCNN(num_nodes, num_edges, device)
        self.device = device
        self.gcnn.to(device)
        
    def _create_weighted_adjacency(self, T0):
        """Creates adjacency matrix from free-flow time (Eq. 1)"""
        weights = 1 / (T0 + 1e-6)
        mask = torch.eye(T0.size(-1), device=self.device).expand_as(T0)
        return weights * (1 - mask)
    
    def train(self, dataset, epochs=100, lr=0.01):
        optimizer = torch.optim.Adam(self.gcnn.parameters(), lr=lr)
        
        for epoch in range(epochs):
            total_loss = 0
            for batch in dataset:
                X = batch['od_matrix'].to(self.device)
                T0 = batch['free_flow_time'].to(self.device)
                C = batch['capacity'].to(self.device)  # [batch, num_edges]
                y_true = batch['link_flows'].to(self.device)

                A_w = self._create_weighted_adjacency(T0)
                y_pred = self.gcnn(X, A_w, C)
                
                # Composite loss:
                # 1. MSE for flow accuracy
                batch_size = C.size(0)
                adj_mask = (C > 0).float()
                y_true_links = y_true.view(batch_size, -1)
                adj_mask = adj_mask.view(batch_size, -1)
                y_true_links = y_true_links[adj_mask.bool()].view(batch_size, 76)

                mse_loss = F.mse_loss(y_pred, y_true_links)
                
                C_links = C.view(batch_size, -1)  # [batch, N*N]
                # 3. Filter only existing links (non-zero capacities)
                link_capacities = C_links[adj_mask.bool()].view(batch_size, 76)
                # 2. Capacity violation penalty (using ReLU to penalize only over-capacity)
                cap_violation = F.relu(y_pred - link_capacities).mean()  # Mean over-capacity
                
                # Combined loss (weighted)
                loss = mse_loss + 10.0 * cap_violation  # Adjust weight as needed
                
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                
                total_loss += loss.item()
            
            print(f"Epoch {epoch+1}, Loss: {total_loss/len(dataset):.4f} | "
                  f"MSE: {mse_loss.item():.4f}, CapViol: {cap_violation.item():.4f}")

class TrafficDataset(torch.utils.data.Dataset):
    """
    Dataset class for traffic assignment problem
    """
    def __init__(self, od_matrices, capacities, free_flow_times, link_flows):
        """
        Args:
            od_matrices: List/array of OD demand matrices [num_samples, num_nodes, num_nodes]
            capacities: List/array of capacity matrices [num_samples, num_edges]
            free_flow_times: List/array of free-flow time matrices [num_samples, num_nodes, num_nodes]
            link_flows: List/array of ground truth link flows [num_samples, num_edges]
        """
        self.nodes_num = len(od_matrices)
        self.od_matrices = torch.tensor(od_matrices, dtype=torch.float32)
        self.capacities = torch.tensor([capacities for _ in range(self.nodes_num)], dtype=torch.float32)
        self.free_flow_times = torch.tensor([free_flow_times for _ in range(self.nodes_num)], dtype=torch.float32)
        self.link_flows = torch.tensor(link_flows, dtype=torch.float32)
        
    def __len__(self):
        return len(self.od_matrices)
    
    def __getitem__(self, idx):
        return {
            'od_matrix': self.od_matrices[idx],
            'capacity': self.capacities[idx],
            'free_flow_time': self.free_flow_times[idx],
            'link_flows': self.link_flows[idx]
        }
    
    def get_batches(self, batch_size):
        dataloader = torch.utils.data.DataLoader(
            self, batch_size=batch_size, shuffle=True
        )
        return dataloader

In [35]:
sioux = create_network_df(network_name="SiouxFalls")
T_0, C = prepare_network_data(sioux)

In [84]:
directory = "/home/podozerovapo/traffic_assignment/data/sioux/uncongested"

inputs = []
outputs = []
metadata = []

for filename in sorted(os.listdir(directory)):
    if filename.endswith(".pkl"):
        filepath = os.path.join(directory, filename)
        
        with open(filepath, 'rb') as f:
            data_pair = pickle.load(f)
            
            inputs.append(data_pair['input'])
            outputs.append(data_pair['output'])
            metadata.append(data_pair.get('metadata', None))

input_matrices = np.array(inputs)  # [num_samples, num_nodes, num_nodes]
output_matrices = np.array(outputs)  # [num_samples, num_nodes, num_nodes]

print(f"Loaded {len(inputs)} samples")
print(f"Input shape: {input_matrices.shape}")
print(f"Output shape: {output_matrices.shape}")

Loaded 1000 samples
Input shape: (1000, 24, 24)
Output shape: (1000, 24, 24)


In [85]:
dataset_full = TrafficDataset(input_matrices, C, T_0, output_matrices)
dataset = dataset_full.get_batches(batch_size=128)
num_nodes = len(dataset_full[0]['od_matrix'])
num_edges = (dataset_full[0]['capacity'] > 0).sum()

In [111]:
model = TrafficAssignmentModel(num_nodes, num_edges, device='cuda')

In [114]:
model.train(dataset=dataset, epochs=1000, lr=0.01)

Epoch 1, Loss: 200097.0762 | MSE: 201350.3438, CapViol: 0.0000
Epoch 2, Loss: 200090.2949 | MSE: 201204.4844, CapViol: 0.0000
Epoch 3, Loss: 200189.6875 | MSE: 205477.1406, CapViol: 0.0000
Epoch 4, Loss: 200140.1973 | MSE: 203377.1094, CapViol: 0.0000
Epoch 5, Loss: 200117.4023 | MSE: 202423.9688, CapViol: 0.0000
Epoch 6, Loss: 200023.5781 | MSE: 198410.6562, CapViol: 0.0000
Epoch 7, Loss: 200171.4355 | MSE: 204710.6719, CapViol: 0.0000
Epoch 8, Loss: 200084.7656 | MSE: 201020.2500, CapViol: 0.0000
Epoch 9, Loss: 200161.1797 | MSE: 204280.4688, CapViol: 0.0000
Epoch 10, Loss: 200078.8867 | MSE: 200776.8750, CapViol: 0.0000
Epoch 11, Loss: 200028.0977 | MSE: 198606.9062, CapViol: 0.0000
Epoch 12, Loss: 200053.5098 | MSE: 199686.1250, CapViol: 0.0000
Epoch 13, Loss: 200060.0664 | MSE: 199969.5938, CapViol: 0.0000
Epoch 14, Loss: 200023.9121 | MSE: 198431.0938, CapViol: 0.0000
Epoch 15, Loss: 199928.7559 | MSE: 194385.2656, CapViol: 0.0000
Epoch 16, Loss: 199993.5488 | MSE: 197143.1562, C

In [93]:
9728 / 128

76.0

In [ ]:
def predict_flows(model, test_sample):
    with torch.no_grad():
        A_w = model._create_weighted_adjacency(test_sample['free_flow_time'])
        predicted_flows = model.gcnn(
            test_sample['od_matrix'].to('cpu'),
            A_w,
            test_sample['capacity'].to('cpu')
        )
    
    return predicted_flows.squeeze(0).numpy()
predicted = predict_flows(model, dataset_full[0])

AttributeError: 'TrafficAssignmentModel' object has no attribute 'to'

In [ ]:
flows = model.gcnn()